In [2]:
import pandas as pd
import numpy as np
import plotly
import importlib
import asana
import wartsila_asana_utils as wa

import plotly.express as px

In [3]:
importlib.reload(wa)

<module 'wartsila_asana_utils' from 'C:\\Users\\dha042\\Documents\\W_Development\\Resource_Planning\\wartsila_asana_utils.py'>

# Refactoring the resource Analysis

In [ ]:
### Initial Data PreProcessing 
def prepare_data_for_analysis(df_in, use_pj_groups=True)
    """
    Input
    -----
    df_in : dataframe - preformatted tasks from asan api
    use_pj_groups : apply groupings to colocated Wartsila Projects
    
    Returns
    -------
    
    
    """

    if use_pj_groups:
        df_tasks = make_project_groups(df_in.copy())
        print('using groups')
    else:
        df_tasks = df_in.copy()
   
    ## Unassigned - Cx
    # get all cx 'unassigned' tasks (denoted by 'Antti Pitkanen' as assignee)
    df_t = df_tasks
    df_t = df_t[df_t['assignee_name']=='Antti Pitkänen']

    ## Reference Cx Personnel
    pdb_list = wa.get_persondb_api_data()
    df_personnel_master = wa.make_persondb_df(pdb_list)
    df_p_all = df_personnel_master.copy()

    cx_all_persons = df_p_all[df_p_all['role'].isin(['Commissioning Manager', 'Commissioning Engineer'])].sort_values('region')
    cx_all_persons_count = cx_all_persons['person_name'].nunique()
    list_all_cx_persons = cx_all_persons['person_name'].tolist()

    ## Assigned - Cx
    # Get tasks assigned to Cx Managers or Cx Engineers
    roles_list = ['Commissioning Manager', 'Commissioning Engineer']
    df_ass = df_tasks.copy()
    df_ass = df_ass[df_ass['role'].isin(roles_list)]

    # exclude unassigned tasks
    df_ass = df_tasks.copy()
    exclude_assignee_list = ['Antti Pitkänen', 'unassigned']
    df_ass = df_ass[~df_ass['assignee_name'].isin(exclude_assignee_list)]

    # verify assignees against master list
    # todo determine if we need to add region filter
    df_ass = df_ass[df_ass['assignee_name'].isin(cx_all_persons['person_name'].to_list())]

    ## Assigned and Unassigned 
    df_cx_all = pd.concat([df_t, df_ass])
    
    return df_cx_all, df_t, df_ass

In [ ]:
### Applying Analysis Functions 
    ## return values

### Create Plots